In [1]:
# required Python imports
import pandas as pd
import numpy as np
from types import SimpleNamespace

import ftir_funct as f
np.set_printoptions(suppress=True)

module FTIR v.2024.3.19 imported


## Generate database

In [2]:
f.explore_Euler_space(step=7, lower_bounds=(0, 0, 0), upper_bounds=(89, 89, 179))

array([[  0,   0,   0],
       [  0,   0,   7],
       [  0,   0,  14],
       ...,
       [ 84,  84, 161],
       [ 84,  84, 168],
       [ 84,  84, 175]])

In [3]:
# generate Euler angles each 7 degrees and store in the database
database = SimpleNamespace(euler=f.explore_Euler_space(step=7, lower_bounds=(0, 0, 0), upper_bounds=(89, 89, 179)))
database.euler.shape

(4394, 3)

In [4]:
# Transmission values for lambda 1987.29
Ta = 0.009882
Tb = 0.995815
Tc = 0.596951

# Generate a mesh of values defining the reference transmissión envelope
polar, azimuths = f.regular_S2_grid(n_squared=500)
T = f.Tvalues(trans=(Ta, Tb, Tc), azimuth=azimuths, polar=polar)
x, y, z = f.sph2cart(T, azimuths, polar)

# Generate 16 measures at different azimuth angles
angles = np.arange(0, 360, 45/2)

# initialize variables
T_vals = []
azi_vals = []

for euler in database.euler:
    # rotate
    x2, y2, z2 = f.rotate(coordinates=(x, y, z), euler_ang=euler)

    # extract XY intersection
    xy_vectors = f.extract_XY_section_fast2(x2, y2, z2)

    # get the indexes of specific angles
    indexes = f.find_nearest(xy_vectors['angles'], angles)

    # append values
    T_vals.append(xy_vectors.loc[indexes, ['T']].T.values.tolist()[0])
    azi_vals.append(xy_vectors.loc[indexes, ['angles']].T.values.tolist()[0])

# store values in database
database.T_values = np.array(T_vals)
database.azimuths = np.array(azi_vals)

In [5]:
# check
print(database.euler.shape)
print(database.T_values.shape)
print(database.azimuths.shape)

(4394, 3)
(4394, 16)
(4394, 16)


In [6]:
# check
print('Euler angles: ', database.euler[-1])
print('T values: ', np.around(database.T_values[-1], 2))
print('Azimuths: ', np.around(database.azimuths[-1], 1))

Euler angles:  [ 84  84 175]
T values:  [0.98 0.89 0.74 0.62 0.6  0.68 0.83 0.96 0.98 0.89 0.74 0.62 0.6  0.68
 0.83 0.96]
Azimuths:  [  0.2  22.5  45.2  67.4  90.  112.3 135.  157.5 180.  202.5 225.2 247.5
 270.  292.3 315.  337.5]


In [7]:
# check
measures = np.column_stack((database.T_values[-1],
                            database.azimuths[-1],
                            np.full_like(database.azimuths[-1], 90)))

estimate_1 = f.find_orientation(measurements=measures, params=(Ta, Tb, Tc), silent=False)
estimate_1.x

Calculated Orientation: [ 84.  84. 175.]


array([ 83.9883221 ,  83.99865909, 175.00345525])

In [8]:
# check
f.calc_misorientation(database.euler[-1], estimate_1.x)

0.012

## Estimate

In [9]:
# initialize variables
default10_algorithm = np.empty(database.euler.shape[0])
default15_algorithm = np.empty(database.euler.shape[0])
default20_algorithm = np.empty(database.euler.shape[0])
default25_algorithm = np.empty(database.euler.shape[0])
default30_algorithm = np.empty(database.euler.shape[0])

for index, orientation in enumerate(database.euler):
    # print('Real:', np.around(orientation, 0))

    measures = np.column_stack((database.T_values[index],
                                database.azimuths[index],
                                np.full_like(database.azimuths[index], 90)))

    estimate_1 = f.find_orientation(measurements=measures, params=(Ta, Tb, Tc), num_guesses=10)
    default10_algorithm[index] = f.calc_disorientation(orientation, estimate_1.x)

    estimate_2 = f.find_orientation(measurements=measures, params=(Ta, Tb, Tc), num_guesses=15)
    default15_algorithm[index] = f.calc_disorientation(orientation, estimate_2.x)

    estimate_3 = f.find_orientation(measurements=measures, params=(Ta, Tb, Tc), num_guesses=20)
    default20_algorithm[index] = f.calc_disorientation(orientation, estimate_3.x)

    estimate_4 = f.find_orientation(measurements=measures, params=(Ta, Tb, Tc), num_guesses=25)
    default25_algorithm[index] = f.calc_disorientation(orientation, estimate_4.x)

    estimate_5 = f.find_orientation(measurements=measures, params=(Ta, Tb, Tc), num_guesses=30)
    default30_algorithm[index] = f.calc_disorientation(orientation, estimate_5.x)


## Generate dataframe and save data

In [10]:
dataset = pd.DataFrame(
    {
        "Euler1": database.euler[:, 0],
        "Euler2": database.euler[:, 1],
        "Euler3": database.euler[:, 2],
        "default10": default10_algorithm,
        "default15": default15_algorithm,
        "default20": default20_algorithm,
        "default25": default25_algorithm,
        "default30": default30_algorithm,
    }
)

dataset

,Euler1,Euler2,Euler3,default10,default20,default30
0,0,0,0,0.000,0.000,0.000
1,0,0,7,0.000,0.276,0.231
2,0,0,14,0.000,0.152,0.000
3,0,0,21,0.247,0.253,0.000
4,0,0,28,0.300,0.000,0.000
...,...,...,...,...,...,...
4389,84,84,147,0.012,0.009,0.012
4390,84,84,154,0.012,0.012,0.010
4391,84,84,161,0.013,0.010,0.009
4392,84,84,168,0.013,0.010,0.010


In [11]:
from datetime import date    
today = date.today().isoformat()

In [12]:
dataset.to_csv(today + '_lamb1987_step7_guesses.csv', index=False)

In [13]:
# quick statistical overview
dataset[['default10', 'default20', 'default30']].describe()

,default10,default20,default30
count,4394.000000,4394.000000,4394.000000
mean,0.129772,0.036369,0.023275
std,0.986069,0.195081,0.071070
min,0.000000,0.000000,0.000000
25%,0.003000,0.003000,0.003000
50%,0.005000,0.004000,0.004000
75%,0.014000,0.011000,0.010000
max,39.621000,6.949000,1.593000
